# 1. Importamos librerias

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import nltk
from nltk import tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Importamos dataSet

In [23]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_trabajo_train = df.copy()
df_trabajo_test = df_test.copy()

df_trabajo_train

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
49995,49995,Pensé que esta película hizo un buen trabajo a...,positivo
49996,49996,"Mala parcela, mal diálogo, mala actuación, dir...",negativo
49997,49997,Soy católica enseñada en escuelas primarias pa...,negativo
49998,49998,Voy a tener que estar en desacuerdo con el com...,negativo


# Preprocesamiento

## Definimos una funcion para quitar letras con tildes

In [24]:
def remove_tildes(text):
    remove = (("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"))

    for a, b in remove:
        text = text.replace(a, b).replace(a.upper(), b.upper())
    return text

## Stop words
- Usaremos stopWords tanto en español como en ingles y les haremos un procesamiento para quitar las tildes.

In [25]:
# Por recomendacion generales de la comunidad, quitaremos las tildes de las stop words.
stop_words = stopwords.words("english") + stopwords.words("spanish")
stop_words = [remove_tildes(word) for word in stop_words]

## Vectorizacion
Para vectorizar utilizaremos TfidfVectorizer ya que por indicaciones de la catedra y pruebas del equipo suele ofrecer una mejora en el rendimiento.

### Parametros de TfidfVectorizer:
1.   **max_features:** Este parámetro especifica el número máximo de características (términos o palabras) que se deben extraer y tener en cuenta. Por ejemplo, en 2000, significa que solo se considerarán las 2000 palabras más frecuentes en el conjunto de documentos. Probamos con utilizar diversos valores en este campo, (1000, 2500, 5000) y el mejor resultado lo conseguimos dejandolo con valores default.

2.   **stop_words:** Este parámetro se utiliza para especificar una lista de palabras comunes que se deben omitir durante el proceso de vectorización. Las palabras "stop words" suelen ser palabras muy comunes en un idioma (como "the", "and", "is", etc.) que no aportan mucha información sobre el contenido del texto y pueden ser eliminadas para mejorar la eficiencia y la relevancia del análisis.

3. **strip_accents:** Este parámetro indica si se deben eliminar o reemplazar acentos y diacríticos durante el preprocesamiento del texto.

In [26]:
vectorizer = TfidfVectorizer(stop_words = stop_words, strip_accents = "unicode")

# 3. Dividir conjunto train y test

In [27]:
x_train, x_test, y_train, y_test = train_test_split(df_trabajo_train.review_es, df_trabajo_train.sentimiento, test_size = 0.30, random_state = 25, shuffle = True)

# 4. Instancio, entreno y testeo modelos

Se entrenaran modelos Naive Bayes Multinomial utilizando una tokenizacion con el *vectorizer* creado anteriormente.

In [28]:
model_tfid = make_pipeline(vectorizer, MultinomialNB())

model_tfid.fit(x_train, y_train)

model_tfid_prediction = model_tfid.predict(x_test)

# 5. Evaluo y visualizo resultados

In [29]:
print("Matrix de confusion de TFID:")
print(confusion_matrix(y_test, model_tfid_prediction))

print()

print("TFID Metrics:")
print(metrics.classification_report(y_test, model_tfid_prediction))


Matrix de confusion de TFID:
[[6516 1083]
 [1074 6327]]

TFID Metrics:
              precision    recall  f1-score   support

    negativo       0.86      0.86      0.86      7599
    positivo       0.85      0.85      0.85      7401

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



Tal y como preveimos, TFI es superior.

# 5.1 Busqueda de los mejores hiperparametros

In [43]:
parameters = {
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'multinomialnb__alpha': [0.1, 0.5, 1.0],
}

optimized = make_pipeline(vectorizer, MultinomialNB())

grid_search = GridSearchCV(optimized, parameters, cv=5, n_jobs=-1)

grid_search.fit(x_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el rendimiento accuracy en el conjunto de prueba
accuracy = grid_search.score(x_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy}")

best_model = grid_search.best_estimator_

Mejores parámetros encontrados:
{'multinomialnb__alpha': 0.1, 'tfidfvectorizer__ngram_range': (1, 2)}
Precisión en el conjunto de prueba: 0.8838


# 6. Pasamos a evaluar con el conjunto test real

In [44]:
dft = pd.read_csv("test.csv")
ds_test = dft.copy()

ds_test_ID = ds_test.ID
ds_test_for_predict = ds_test.review_es

test_prediction = best_model.predict(ds_test_for_predict)

ds_submission = pd.DataFrame({'ID': ds_test['ID'], 'sentimiento': test_prediction})
ds_submission.to_csv('./prediccion_final.csv', index=False)

joblib.dump(best_model, 'bayes_entrenado.joblib')

['bayes_entrenado.joblib']